In [49]:
import pandas as pd
from pandas.tseries.offsets import *
import numpy as np
import glob

In [50]:
# File position
resource = '../../data/'
results = '../../data/generated/'

In [51]:
# Load spreadsheet from Datastream
xl = pd.ExcelFile(resource +'external/datastream.xlsx')

# Load industrial production
ind_prod = xl.parse('ind_prod', skiprows=2, parse_dates=[0])
ind_prod = ind_prod.set_axis(['date', 'ind_prod'], axis=1)
ind_prod['date'] = ind_prod['date'] + MonthEnd(0)

# Load inflation 
# cpi = xl.parse('cpi', skiprows=2, parse_dates=[0])
# cpi = cpi.set_axis(['date', 'cpi'], axis=1)
# cpi['date'] = cpi['date'] + MonthEnd(0)

# Load gold index
gold = xl.parse('gold', skiprows=2, parse_dates=[0])
gold = gold.set_axis(['date', 'gold'], axis=1)
gold['month'] = gold['date'].dt.to_period('M')
gold = gold.groupby('month').last().reset_index(drop=True)
gold['date'] = gold['date'] + MonthEnd(0)

# Merge
datastream = pd.merge(ind_prod, gold, on='date', how='outer')
# ts = pd.merge(ts, cpi, on='date', how='outer')

In [52]:
# Load spreadsheet from Shiller's website
xl = pd.ExcelFile(resource +'external/ie_data.xls')
ie_m = xl.parse('Data', skiprows=7)
ie_m = ie_m[['Date','P','D','CPI']]
ie_m = ie_m.set_axis(['date','sp_price','sp_div','cpi'], axis=1)
ie_m['date'] =  ie_m['date'].apply(lambda x: f'{x:.2f}')
ie_m['date'] = pd.to_datetime(ie_m['date'], format='%Y.%m')
ie_m['date'] = ie_m['date'] + MonthEnd(0)
ie_m = ie_m.iloc[:-1]

# Merge
ts = pd.merge(ie_m, datastream, on='date', how='outer')

In [53]:
# Load 3-factor data from kenneth frech website
ff_m = pd.read_csv(resource +'external/F-F_Research_Data_Factors.CSV', skiprows=3)
ff_m = ff_m.set_axis(['date','ex_mkt','smb','hml','rf'], axis=1)
ff_m = ff_m.loc[:ff_m[ff_m['date'] == ' Annual Factors: January-December '].index[0]-1]
ff_m['date'] = pd.to_datetime(ff_m['date'], format='%Y%m')
ff_m['date'] = ff_m['date'] + MonthEnd(0)
ff_m[['ex_mkt','smb','hml','rf']] = ff_m[['ex_mkt','smb','hml','rf']].apply(pd.to_numeric)
ff_m[['ex_mkt','smb','hml','rf']] = ff_m[['ex_mkt','smb','hml','rf']].div(100)

# Merge
ts = pd.merge(ts, ff_m, on='date', how='left')

In [54]:
# Load momentum data from kenneth frech website
mom_m = pd.read_csv(resource +'external/F-F_Momentum_Factor.CSV', skiprows=13)
mom_m = mom_m.set_axis(['date','mom'], axis=1)
mom_m = mom_m.loc[:mom_m[mom_m['date'] == 'Annual Factors:'].index[0]-1]
mom_m['date'] = pd.to_datetime(mom_m['date'], format='%Y%m')
mom_m['date'] = mom_m['date'] + MonthEnd(0)
mom_m['mom'] = mom_m['mom'].apply(pd.to_numeric)
mom_m['mom'] = mom_m['mom']/100

# Merge
ts = pd.merge(ts, mom_m, on='date', how='left')

In [55]:
# Load data from fred
fred = pd.read_csv(resource +'pulled/fred.csv', parse_dates=['date'])
fred = fred[['date','DTB3','DGS10','DGS1','AAA','BAA','TWEXM','DTWEXAFEGS']]
fred['TWEXM'] = pd.to_numeric(fred['TWEXM'], errors='coerce')
fred['DTWEXAFEGS'] = pd.to_numeric(fred['DTWEXAFEGS'], errors='coerce')
fred[['DTB3','DGS10','DGS1','AAA','BAA','TWEXM','DTWEXAFEGS']] = fred[['DTB3','DGS10','DGS1','AAA','BAA','TWEXM','DTWEXAFEGS']].div(100)

# Merge
ts = pd.merge(ts, fred, on='date', how='left')

In [56]:
# Load bond returns data (high yield and government) from bloomBMrg
xl = pd.ExcelFile(resource +'external/bloomberg.xlsx')

# Load industrial production
bl = xl.parse('data', skiprows=2, parse_dates=['Dates'])
bl = bl.set_axis(['date', 'long_gov_ret', 'medium_gov_ret','high_yd_bd_ret'], axis=1)
bl[['long_gov_ret', 'medium_gov_ret','high_yd_bd_ret']] = bl[['long_gov_ret', 'medium_gov_ret','high_yd_bd_ret']].div(100)
bl['date'] = bl['date'] + MonthEnd(0)

# Merge
ts = pd.merge(ts, bl, on='date', how='left')

In [57]:
# Load Vassalou dsv (change in the aggregate survival probability) data
xl = pd.ExcelFile(resource +'external/DSV.xls')

# Load (Vassalou) dsv data
vass = xl.parse('Sheet1', header=None)
vass = vass.set_axis(['date', 'v_dsv'], axis=1)
vass['date'] = pd.to_datetime(vass['date'], format='%Y%m')
vass['date'] = vass['date'] + MonthEnd(0)
vass['v_dsv'] = vass['v_dsv']/100

# Merge
ts = pd.merge(ts, vass, on='date', how='left')

In [58]:
# Load my own default prob data
files = glob.glob(resource + 'generated/p_def_*.csv')
data = []
for file in files:
    df = pd.read_csv(file, parse_dates=['date'])
    data.append(df)
default = pd.concat(data)
default.reset_index(drop=True, inplace=True)

# default['month']=default['date'].dt.month
# default['year']=default['date'].dt.year
default = default.groupby(['date'])['p_def'].mean().reset_index()
default['p_surv'] = 1 - default['p_def']
default['my_dsv'] = (default['p_surv'] - default['p_surv'].shift(1))

my_dsv = default[['date','my_dsv']]

# Merge
ts = pd.merge(ts, my_dsv, on='date', how='left')

In [59]:
ts.to_csv(results + 'time_series.csv', index=False)

In [60]:
# Load BM portfolios returns data from kenneth frech website
bm_port = pd.read_csv(resource +'external/Portfolios_Formed_on_BE-ME.CSV', skiprows=23)
bm_port = bm_port.set_axis(['date','less_zero','lo_30','med_40','hi_30','lo_20','qnt_2','qnt3','qnt_4','hi_20','dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10'], axis=1)
bm_port = bm_port.loc[:bm_port[bm_port['date'] == '  Equal Weight Returns -- Monthly'].index[0]-2]
bm_port['date'] = pd.to_datetime(bm_port['date'], format='%Y%m')
bm_port['date'] = bm_port['date'] + MonthEnd(0)
bm_port[['less_zero','lo_30','med_40','hi_30','lo_20','qnt_2','qnt3','qnt_4','hi_20','dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10']] = bm_port[['less_zero','lo_30','med_40','hi_30','lo_20','qnt_2','qnt3','qnt_4','hi_20','dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10']].apply(pd.to_numeric)
bm_port[['less_zero','lo_30','med_40','hi_30','lo_20','qnt_2','qnt3','qnt_4','hi_20','dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10']] = bm_port[['less_zero','lo_30','med_40','hi_30','lo_20','qnt_2','qnt3','qnt_4','hi_20','dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10']].div(100)

# Merge
bm_port.to_csv(results + 'bm_port.csv', index=False)

In [61]:
# Load mom portfolios returns data from kenneth frech website
size_port = pd.read_csv(resource +'external/Portfolios_Formed_on_ME.CSV', skiprows=12)
size_port = size_port.set_axis(['date','less_zero','lo_30','med_40','hi_30','lo_20','qnt_2','qnt3','qnt_4','hi_20','dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10'], axis=1)
size_port = size_port.loc[:size_port[size_port['date'] == '  Equal Weight Returns -- Monthly'].index[0]-2]
size_port['date'] = pd.to_datetime(size_port['date'], format='%Y%m')
size_port['date'] = size_port['date'] + MonthEnd(0)
size_port[['less_zero','lo_30','med_40','hi_30','lo_20','qnt_2','qnt3','qnt_4','hi_20','dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10']] = size_port[['less_zero','lo_30','med_40','hi_30','lo_20','qnt_2','qnt3','qnt_4','hi_20','dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10']].apply(pd.to_numeric)
size_port[['less_zero','lo_30','med_40','hi_30','lo_20','qnt_2','qnt3','qnt_4','hi_20','dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10']] = size_port[['less_zero','lo_30','med_40','hi_30','lo_20','qnt_2','qnt3','qnt_4','hi_20','dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10']].div(100)

# Merge
size_port.to_csv(results + 'size_port.csv', index=False)

In [62]:
# Load momentum portfolios returns data from kenneth frech website
mom_port = pd.read_csv(resource +'external/10_Portfolios_Prior_12_2.CSV', skiprows=10)
mom_port = mom_port.set_axis(['date','dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10'], axis=1)
mom_port = mom_port.loc[:mom_port[mom_port['date'] == '  Average Equal Weighted Returns -- Monthly'].index[0]-2]
mom_port['date'] = pd.to_datetime(mom_port['date'], format='%Y%m')
mom_port['date'] = mom_port['date'] + MonthEnd(0)
mom_port[['dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10']] = mom_port[['dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10']].apply(pd.to_numeric)
mom_port[['dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10']] = mom_port[['dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10']].div(100)

# Merge
mom_port.to_csv(results + 'mom_port.csv', index=False)